# Week 1 homework: Ride duration prediction for Yellow Taxi Trip Records

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in [this notebook](week1_duration-prediction.ipynb), but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

## Imports

In [ ]:
import pandas as pd

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

## Data

Taken from the page: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

Downloading "Yellow Taxi Trip Records" for January and February, 2023.

In [ ]:
# Yellow Taxi Trip Records for January, 2023
# !wget -O data/yellow_tripdata_2023-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

# Yellow Taxi Trip Records for February, 2023
# !wget -O data/yellow_tripdata_2023-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

## Q1

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [ ]:
df_january = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")

df_january

In [ ]:
df_january.shape

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [ ]:
df_january.columns

In [ ]:
df_january["duration"] = (
    df_january.tpep_dropoff_datetime - df_january.tpep_pickup_datetime
)
df_january.duration = df_january.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
df_january.duration.std()

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [ ]:
rows_originally = df_january.shape[0]

In [ ]:
df_january = df_january[(df_january.duration >= 1) & (df_january.duration <= 60)]

rows_filtered = df_january.shape[0]

In [ ]:
rows_filtered / rows_originally

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [ ]:
categorical = ["PULocationID", "DOLocationID"]
df_january[categorical] = df_january[categorical].astype(str)

In [ ]:
train_dicts = df_january[categorical].to_dict(orient="records")

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train.shape

In [ ]:
target = "duration"
y_train = df_january[target].values

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where `duration` is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred = lr.predict(X_train)

In [ ]:
root_mean_squared_error(y_train, y_pred)

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [ ]:
df_february = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")

df_february.shape

In [ ]:
df_february["duration"] = (
    df_february.tpep_dropoff_datetime - df_february.tpep_pickup_datetime
)
df_february.duration = df_february.duration.apply(lambda td: td.total_seconds() / 60)

In [ ]:
# In real life, we would not remove outliers,
# because it is a testset and we don't know the outliers in advance
# (in real life we won't have a target)
# But here it seems to be necessary

df_february = df_february[(df_february.duration >= 1) & (df_february.duration <= 60)]

In [ ]:
df_february[categorical] = df_february[categorical].astype(str)
test_dicts = df_february[categorical].to_dict(orient="records")

In [ ]:
X_test = dv.transform(test_dicts)

In [ ]:
y_test = df_february[target].values

In [ ]:
y_test_pred = lr.predict(X_test)

In [ ]:
root_mean_squared_error(y_test, y_test_pred)

## Submit the results

The results are submitted here: https://courses.datatalks.club/mlops-zoomcamp-2025/homework/hw1
